In [1]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
db = client['stock_data']
db.list_collections()
db.list_collection_names()

['AMZN']

In [4]:
collection = 'AMZN'
db.drop_collection(collection)

{'nIndexesWas': 1, 'ns': 'stock_data.AMZN', 'ok': 1.0}

In [5]:
import yfinance as yf

ticker = 'AMZN'
df = yf.download(ticker)
df.tail()


[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-02-10,2085.01,2135.60,2084.96,2133.91,2133.91,5056200
2020-02-11,2150.90,2185.95,2136.00,2150.80,2150.80,5746000
2020-02-12,2163.20,2180.25,2155.29,2160.00,2160.00,3334300
2020-02-13,2144.99,2170.28,2142.00,2149.87,2149.87,3031800
2020-02-14,2155.68,2159.04,2125.89,2134.87,2134.87,2599900


In [1]:
import json
records = json.loads(df.to_json())
print(type(records))
for record in records:
    print(record)
db.myCollection.insert_many(records)
df.T.to_json()

NameError: name 'df' is not defined

In [14]:
list(db.myCollections.find())

[]

In [12]:
df.reset_index(inplace=True)
data_dict = df.to_dict("records")
company= db["Company"]
#company.insert_one({"index":"Sensex","data":data_dict})
company.insert_many(data_dict)

In [13]:
#company.delete_many({'index':'Sensex'})
list(company.find())

[{'_id': ObjectId('5e49c3b6f22afed358100d65'),
  'Date': datetime.datetime(1997, 5, 15, 0, 0),
  'Open': 2.44,
  'High': 2.5,
  'Low': 1.93,
  'Close': 1.96,
  'Adj Close': 1.96,
  'Volume': 72156000},
 {'_id': ObjectId('5e49c3b6f22afed358100d66'),
  'Date': datetime.datetime(1997, 5, 16, 0, 0),
  'Open': 1.97,
  'High': 1.98,
  'Low': 1.71,
  'Close': 1.73,
  'Adj Close': 1.73,
  'Volume': 14700000},
 {'_id': ObjectId('5e49c3b6f22afed358100d67'),
  'Date': datetime.datetime(1997, 5, 19, 0, 0),
  'Open': 1.76,
  'High': 1.77,
  'Low': 1.62,
  'Close': 1.71,
  'Adj Close': 1.71,
  'Volume': 6106800},
 {'_id': ObjectId('5e49c3b6f22afed358100d68'),
  'Date': datetime.datetime(1997, 5, 20, 0, 0),
  'Open': 1.73,
  'High': 1.75,
  'Low': 1.64,
  'Close': 1.64,
  'Adj Close': 1.64,
  'Volume': 5467200},
 {'_id': ObjectId('5e49c3b6f22afed358100d69'),
  'Date': datetime.datetime(1997, 5, 21, 0, 0),
  'Open': 1.64,
  'High': 1.65,
  'Low': 1.38,
  'Close': 1.43,
  'Adj Close': 1.43,
  'Volume':

In [15]:
df.reset_index(inplace=True)
data_dict = df.to_dict(orient='records')  # Here's our added param..
data_dict[0]


{'index': 0,
 'Date': Timestamp('1997-05-15 00:00:00'),
 'Open': 2.44,
 'High': 2.5,
 'Low': 1.93,
 'Close': 1.96,
 'Adj Close': 1.96,
 'Volume': 72156000}

In [16]:
post = db[ticker]
post.insert_many(list(data_dict))

In [17]:

db.list_collection_names()

['Company', 'AMZN']

In [81]:
import datetime
for c in post.find({'Open': 1801.01}):
    print(c)
for c in post.find({'Date': datetime.datetime(2019, 11, 4, 0, 0)}):
    print(c)

{'_id': ObjectId('5dc77063d7daf9b772e559ca'), 'level_0': 5655, 'index': 5655, 'Date': datetime.datetime(2019, 11, 4, 0, 0), 'Open': 1801.01, 'High': 1815.06, 'Low': 1801.01, 'Close': 1804.66, 'Adj Close': 1804.66, 'Volume': 2771900}
{'_id': ObjectId('5dc77063d7daf9b772e559ca'), 'level_0': 5655, 'index': 5655, 'Date': datetime.datetime(2019, 11, 4, 0, 0), 'Open': 1801.01, 'High': 1815.06, 'Low': 1801.01, 'Close': 1804.66, 'Adj Close': 1804.66, 'Volume': 2771900}


In [15]:
import pandas as pd
from bson import json_util, ObjectId
from pandas.io.json import json_normalize
import json

def mongo_to_dataframe(mongo_data):

        sanitized = json.loads(json_util.dumps(mongo_data))
        normalized = json_normalize(sanitized)
        df = pd.DataFrame(normalized)

        return df

mongo_to_dataframe(post.find({'Open': 1801.01}))


,Adj Close,Close,High,Low,Open,Volume,_id.$oid
0,1804.66,1804.66,1815.06,1801.01,1801.01,2771900,5dc75e71d7daf9b772e51773


In [71]:
mongo_to_dataframe(post.find())

""
